# 数据对接

In [52]:
%%time
# coding=utf8
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
# from cma.music.DataQueryClient import DataQueryClient
import json

class station_zdz():
    def __init__(self,city_code,start,end):
        #self.rs = redis.Redis(host='127.0.0.1', port=6379)
#         self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.city_code = city_code
        self.start = start
        self.end = end
        self.data = self.comupt_city_csv(city_code,start,end)
        self.city_codes = ["331000"]
    def comput_city(self,city_code,start,end):
        """快报或者统计数据的接口"""
        # getSurfEleInRegionByTime
        date_start = dtt.datetime.strptime(start,'%Y-%m-%d %H:%M:%S') 
        date_end = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-60*8)
        label_start = (date_start + offset).strftime('%Y%m%d%H%M') + "00"
        label_end = (date_end + offset).strftime('%Y%m%d%H%M') + "00"   
        labels = "[" + label_start + "," + label_end + "]"
        #timelabel = self.decode_time_str(timesdelay)
        client = DataQueryClient()
        interfaceId = "statSurfEleInRegion"  
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'adminCodes':"331000",
            'elements':"Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,",
            'timeRange':labels,
            'statEles':'MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi',
            #'orderBy':"MAX_WIN_S_Inst_Max:desc",
            'staLevels':"011,012,013,014,015,016", # 12国家站 14区域站
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        #rint(result)
        result_json = json.loads(result)
        clomns = "Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi".split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = "Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave".split(",")
        return data
    def comupt_city_csv(self,city_code,start,end):
        data = pd.read_csv("downfile/comput.csv")
        return data
    def text_rain(self):
        bins=[0,30,40,50,80,100,150,200,300,500,2000]
        labels=['≥0毫米','≥30毫米','≥40毫米','≥50毫米','≥80毫米','≥100毫米','≥150毫米','≥200毫米','≥300毫米','≥500毫米']
        rain = data[(data['rain']>0) & (data['rain']<5009)]
        rain['rank']=pd.cut(rain['rain'],bins,right=False,labels=labels)
        del rain['Unnamed: 0'] # 天擎不需要
        rain.reset_index(drop=True)
        if city_code in self.citys:
            cnty = rain.groupby(['Cnty'])['rain'].mean().to_dict()
        else:
            cnty = rain.groupby(['Town'])['rain'].mean().to_dict()
        rain_max = rain.sort_values(by="rain",ascending=False).head(10).to_dict()
        rain_hour = rain.sort_values(by="rainhour",ascending=False).head(5).to_dict()
        town_max = rain.groupby(['Town','Cnty'])['rain'].max().sort_values(ascending=False).head(10)
        town_max_str = []
        for i in town_max.index.to_list():
            single = (i[0] +"-" +  i[1]+ ":" +str(town_max[i]) + "毫米")
            town_max_str.append(single)
        rank_station = {
            '≥0毫米':len(rain[(rain['rain']>0) & (rain['rain']<30)].value_counts()),
            '≥30毫米':len(rain[(rain['rain']>=30) & (rain['rain']<40)].value_counts()),
            '≥40毫米':len(rain[(rain['rain']>=40) & (rain['rain']<50)].value_counts()),
            '≥50毫米':len(rain[(rain['rain']>=50) & (rain['rain']<80)].value_counts()),
            '≥80毫米':len(rain[(rain['rain']>=80) & (rain['rain']<100)].value_counts()),
            '≥100毫米':len(rain[(rain['rain']>=100) & (rain['rain']<150)].value_counts()),
            '≥150毫米':len(rain[(rain['rain']>=150) & (rain['rain']<200)].value_counts()),
            '≥200毫米':len(rain[(rain['rain']>=200) & (rain['rain']<300)].value_counts()),
            '≥300毫米':len(rain[(rain['rain']>=300) & (rain['rain']<500)].value_counts()),
            '≥500毫米':len(rain[(rain['rain']>=500) & (rain['rain']<2000)].value_counts())
        }
        # 乡镇个数统计
        town_group = rain.groupby(['Town'])['rain'].max()  
        dict_town = {'Town':town_group.index,'rain':town_group.values}
        town_range = pd.DataFrame(data =dict_town)
        town_range['rank']=pd.cut(town_range['rain'],bins,right=False,labels=labels) 
        rank_town = {
            '≥0毫米':len(town_range[(town_range['rain']>0) & (town_range['rain']<30)].value_counts()),
            '≥30毫米':len(town_range[(town_range['rain']>=30) & (town_range['rain']<40)].value_counts()),
            '≥40毫米':len(town_range[(town_range['rain']>=40) & (town_range['rain']<50)].value_counts()),
            '≥50毫米':len(town_range[(town_range['rain']>=50) & (town_range['rain']<80)].value_counts()),
            '≥80毫米':len(town_range[(town_range['rain']>=80) & (town_range['rain']<100)].value_counts()),
            '≥100毫米':len(town_range[(town_range['rain']>=100) & (town_range['rain']<150)].value_counts()),
            '≥150毫米':len(town_range[(town_range['rain']>=150) & (town_range['rain']<200)].value_counts()),
            '≥200毫米':len(town_range[(town_range['rain']>=200) & (town_range['rain']<300)].value_counts()),
            '≥300毫米':len(town_range[(town_range['rain']>=300) & (town_range['rain']<500)].value_counts()),
            '≥500毫米':len(town_range[(town_range['rain']>=500) & (town_range['rain']<2000)].value_counts())
        }
        rain_dir = {
            "average_city":cnty,
            "station_index":rain_max,
            "hour_max":rain_hour,
            "town_max":town_max_str,
            "station_count":rank_station,
            "town_count":rank_town        
        }
        return rain_dir
    def text_wind(self):
        orig = self.data
        data = orig.sort_values(by="wind",ascending=False)#.head(5).to_dict()
        ## 数据分级
        bins=[0,10.7,13.8,17.1,20.7,24.4,28.4,32.6,36.9,41.4,46.1,50.9,56,80]
        labels=['6级风以下','6级风','7级风','8级风','9级风','10级风','11级风','12级风','13级风','14级风','15级风','16级风','17级风']
        wind = data[(data['wind']>10.7) & (data['wind']<5009)]
        wind['rank']=pd.cut(wind['wind'],bins,right=False,labels=labels)
        del wind['Unnamed: 0'] # 天擎不需要
        wind.reset_index(drop=True)# 天擎不需要
        # 获取单站较大的有
        testframe = wind.head(5) 
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['wind']) + "米/秒，"
                #print(index,row['City'],row['Cnty'],row['Town'],row['wind'])
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['wind']) + "米/秒，"
                #print(index,row['City'],row['Cnty'],row['Town'],row['wind'])
        indextext = indextext[:-1] + "。"
        rank_station = {
            '17级及以上风':len(wind[(wind['wind']>=56.4) & (wind['wind']<80)].value_counts()),
            '16级风':len(wind[(wind['wind']>=50.9) & (wind['wind']<56.4)].value_counts()),
            '15级风':len(wind[(wind['wind']>=46.1) & (wind['wind']<50.9)].value_counts()),
            '14级风':len(wind[(wind['wind']>=41.4) & (wind['wind']<46.1)].value_counts()),
            '13级风':len(wind[(wind['wind']>=36.9) & (wind['wind']<41.4)].value_counts()),
            '12级风':len(wind[(wind['wind']>=32.6) & (wind['wind']<36.9)].value_counts()),
            '11级风':len(wind[(wind['wind']>=28.4) & (wind['wind']<32.6)].value_counts()),
            '10级风':len(wind[(wind['wind']>=24.4) & (wind['wind']<28.4)].value_counts()),
            '9级风':len(wind[(wind['wind']>=20.7) & (wind['wind']<24.4)].value_counts()),
            '8级风':len(wind[(wind['wind']>=17.1) & (wind['wind']<20.7)].value_counts()),
            '7级风':len(wind[(wind['wind']>=13.8) & (wind['wind']<17.1)].value_counts()),
            '6级风':len(wind[(wind['wind']>=10.7) & (wind['wind']<13.8)].value_counts()),
            '6级风以下':len(wind[(wind['wind']>0) & (wind['wind']<10.7)].to_dict())
        }
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0:
                numbertext = numbertext + key +"有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['17级及以上风'] >0 :
            text = "【风力通报】 全市出现17级及以上大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['16级风'] >0 :
            text = "【风力通报】 全市出现16级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['15级风'] >0 :
            text = "【风力通报】 全市出现15级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['14级风'] >0 :
            text = "【风力通报】 全市出现14级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['13级风'] >0 :
            text = "【风力通报】 全市出现13级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['12级风'] >0 :
            text = "【风力通报】 全市出现12级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['11级风'] >0 :
            text = "【风力通报】 全市出现11级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['10级风'] >0 :
            text = "【风力通报】 全市出现10级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['9级风'] >0 :
            text = "【风力通报】 全市出现7～9级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['8级风'] >0 :
            text = "【风力通报】 全市出现6～8级大风，风力较大的有："
            text = text + indextext + "其中，"  + numbertext
        else:
            text = ""
        return text
    def text_view(self):
        orig = self.data
        data = orig.sort_values(by="view",ascending=True)#.head(5).to_dict()
        bins=[0,50,200,500,99999]
        labels=['强浓雾','浓雾','大雾','正常']
        view = data[(data['view']>0) & (data['view']<1000)]
        view['rank']=pd.cut(view['view'],bins,right=False,labels=labels)
        del view['Unnamed: 0'] # 天擎不需要
        view.reset_index(drop=True)# 天擎不需要
        rank_station = {
           '强浓雾':len(view[(view['view']>0) & (view['view']<50)].to_dict()),
            '浓雾':len(view[(view['view']>=50) & (view['view']<200)].value_counts()),
            '大雾':len(view[(view['view']>=200) & (view['view']<500)].value_counts()),
            '正常':len(view[(view['view']>=500) & (view['view']<99990)].value_counts()) 
        }
        # 获取单站较大的有
        testframe = view.head(5) 
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['view']) + "米，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['view']) + "米，"
        indextext = indextext[:-1] + "。"
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0 and key!="正常":
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['强浓雾'] >0 :
            text = "【能见度】 全市出现低于50米的强浓雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['浓雾'] >0 :
            text = "【能见度】 全市出现低于200米的浓雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['大雾'] >0 :
            text = "【能见度】 全市出现低于500米的大雾，能见度较低的有："
            text = text + indextext + "其中，"  + numbertext
        else:
            text =""
        return text
    def text_tmax(self):
        orig = self.data
        data = orig.sort_values(by="tmax",ascending=False)#.head(5).to_dict()
        bins=[-40,35,37,40,80]
        labels=['正常','35度以上','37度以上','40度以上']
        tmax = data[(data['tmax']>-40) & (data['tmax']<100)]
        tmax['rank']=pd.cut(tmax['tmax'],bins,right=True,labels=labels)
        del tmax['Unnamed: 0'] # 天擎不需要
        tmax.reset_index(drop=True)# 天擎不需要
        rank_station = {
            '40度以上':len(tmax[(tmax['tmax']>=40) & (tmax['tmax']<80)].value_counts()),
            '37度以上':len(tmax[(tmax['tmax']>=37) & (tmax['tmax']<40)].value_counts()),
            '35度以上':len(tmax[(tmax['tmax']>=35) & (tmax['tmax']<37)].value_counts()),
           '正常':len(tmax[(tmax['tmax']>-40) & (tmax['tmax']<35)].to_dict())      
        }
        # 获取单站较大的有
        testframe = tmax.head(5) 
        indextext = ""
        for index,row in testframe.iterrows():
            if self.city_code in self.city_codes:
                indextext = indextext + row['Cnty'] + row['StationName'] + str(row['tmax']) + "℃，"
            else:
                indextext = indextext + row['Town'] + row['StationName'] + str(row['tmin']) + "℃，"
        indextext = indextext[:-1] + "。"
        #  统计各等级风力的个数
        numbertext = ""
        for key, value in rank_station.items():
            if value>0 and key not in ["正常","35度以上"]:
                numbertext = numbertext + key +"的有" + str(value) + "站,"
        numbertext = numbertext[:-1] + "。"
        if rank_station['40度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext + "其中，"  + numbertext
        elif rank_station['37度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext + "其中，"  + numbertext  
        elif rank_station['35度以上'] >0 :
            text = "【高温通报】 全市出现40℃以上的高温，温度较高的有："
            text = text + indextext 
        else:
            text = ""
        return text
    def text_rain(self):
        # 面雨量
        # 单站前三
        # 
        pass

# 气象快报   
city_code = "331000"
start = '2023-07-01 20:52:00'
end = '2023-07-02 20:52:00'
worker = station_zdz(city_code,start,end)
worker.text_wind()
# worker.text_view()
# worker.text_tmax()

CPU times: user 41.6 ms, sys: 0 ns, total: 41.6 ms
Wall time: 44 ms


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'【风力通报】 全市出现10级大风，风力较大的有：天台县下陈胡26.4米/秒，临海市雀儿岙22.5米/秒，路桥区白果22.3米/秒，临海市东矶21.0米/秒，温岭市北港20.5米/秒。其中，10级风有1站,9级风有3站,8级风有10站,7级风有43站,6级风有66站,6级风以下有18站。'

# 并行计算的算法

In [68]:
%%time
import time
from joblib import Parallel, delayed

def square(num):
    if num==1:
        time.sleep(1)  # 模拟耗时的计算操作
    elif num==2:
        time.sleep(2)
    elif num==3:
        time.sleep(3) 
    elif num==4:
        time.sleep(4)
    elif num==5:
        time.sleep(5)
    elif num==6:
        time.sleep(6)
    return num ** 2

if __name__ == '__main__':
    numbers = [1,2,3,4,5,6]
    start_time = time.time()
    # 并行计算每个数字的平方
    results = Parallel(n_jobs=1)(delayed(square)(num) for num in numbers)
    end_time = time.time()
    # 打印计算结果
    print(results)
    print("并行处理时间：", end_time - start_time)

[1, 4, 9, 16, 25, 36]
并行处理时间： 21.022823333740234
CPU times: user 3.14 ms, sys: 1.3 ms, total: 4.44 ms
Wall time: 21 s


# 备份

In [ ]:
%%time
# coding=utf8
import pymysql
import pymssql 
import numpy as np
import pandas as pd
import pickle
import redis
import datetime as dtt
# from cma.music.DataQueryClient import DataQueryClient
import json

class station_zdz():
    def __init__(self,city_code,start,end):
        #self.rs = redis.Redis(host='127.0.0.1', port=6379)
#         self.conn = pymysql.connect(host="127.0.0.1",port=3306,user="root",passwd="tzqxj58660",db="ZJSZDZDB")
        self.userId = "BEHZ_TZSJ_TZSERVICE" 
        self.pwd = "Liyuan3970!@" 
        self.dataFormat = "json"
        self.data = self.comupt_city_csv(city_code,start,end)
    def comput_city(self,city_code,start,end):
        """快报或者统计数据的接口"""
        # getSurfEleInRegionByTime
        date_start = dtt.datetime.strptime(start,'%Y-%m-%d %H:%M:%S') 
        date_end = dtt.datetime.strptime(end,'%Y-%m-%d %H:%M:%S') 
        offset = dtt.timedelta(minutes=-60*8)
        label_start = (date_start + offset).strftime('%Y%m%d%H%M') + "00"
        label_end = (date_end + offset).strftime('%Y%m%d%H%M') + "00"   
        labels = "[" + label_start + "," + label_end + "]"
        #timelabel = self.decode_time_str(timesdelay)
        client = DataQueryClient()
        interfaceId = "statSurfEleInRegion"  
        params = {
            'dataCode':"SURF_CHN_MUL_MIN",  #SURF_CHN_MUL_HOR
            'adminCodes':"331000",
            'elements':"Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,",
            'timeRange':labels,
            'statEles':'MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi',
            #'orderBy':"MAX_WIN_S_Inst_Max:desc",
            'staLevels':"011,012,013,014,015,016", # 12国家站 14区域站
            'limitCnt':"100000000"
        }
        result = client.callAPI_to_serializedStr(self.userId, self.pwd, interfaceId, params, self.dataFormat)
        #rint(result)
        result_json = json.loads(result)
        clomns = "Province,City,Cnty,Town,Lat,Lon,Alti,Station_levl,Station_Name,Station_Id_C,MAX_TEM,MIN_TEM,SUM_PRE,MAX_PRE_1h,MIN_VIS_HOR_1MI,MAX_WIN_S_Gust_Max,MAX_WIN_S_Avg_2mi".split(",")
        data = pd.DataFrame(result_json['DS'],columns=clomns)
        data.columns = "Province,City,Cnty,Town,Lat,Lon,Height,Stationlevl,StationName,IIIII,tmax,tmin,rain,rainhour,view,wind,windave".split(",")
        return data
    def comupt_city_csv(self,city_code,start,end):
        data = pd.read_csv("downfile/comput.csv")
        return data
    def text_rain(self):
        bins=[0,30,40,50,80,100,150,200,300,500,2000]
        labels=['≥0毫米','≥30毫米','≥40毫米','≥50毫米','≥80毫米','≥100毫米','≥150毫米','≥200毫米','≥300毫米','≥500毫米']
        rain = data[(data['rain']>0) & (data['rain']<5009)]
        rain['rank']=pd.cut(rain['rain'],bins,right=False,labels=labels)
        del rain['Unnamed: 0'] # 天擎不需要
        rain.reset_index(drop=True)
        if city_code in self.citys:
            cnty = rain.groupby(['Cnty'])['rain'].mean().to_dict()
        else:
            cnty = rain.groupby(['Town'])['rain'].mean().to_dict()
        rain_max = rain.sort_values(by="rain",ascending=False).head(10).to_dict()
        rain_hour = rain.sort_values(by="rainhour",ascending=False).head(5).to_dict()
        town_max = rain.groupby(['Town','Cnty'])['rain'].max().sort_values(ascending=False).head(10)
        town_max_str = []
        for i in town_max.index.to_list():
            single = (i[0] +"-" +  i[1]+ ":" +str(town_max[i]) + "毫米")
            town_max_str.append(single)
        rank_station = {
            '≥0毫米':len(rain[(rain['rain']>0) & (rain['rain']<30)].value_counts()),
            '≥30毫米':len(rain[(rain['rain']>=30) & (rain['rain']<40)].value_counts()),
            '≥40毫米':len(rain[(rain['rain']>=40) & (rain['rain']<50)].value_counts()),
            '≥50毫米':len(rain[(rain['rain']>=50) & (rain['rain']<80)].value_counts()),
            '≥80毫米':len(rain[(rain['rain']>=80) & (rain['rain']<100)].value_counts()),
            '≥100毫米':len(rain[(rain['rain']>=100) & (rain['rain']<150)].value_counts()),
            '≥150毫米':len(rain[(rain['rain']>=150) & (rain['rain']<200)].value_counts()),
            '≥200毫米':len(rain[(rain['rain']>=200) & (rain['rain']<300)].value_counts()),
            '≥300毫米':len(rain[(rain['rain']>=300) & (rain['rain']<500)].value_counts()),
            '≥500毫米':len(rain[(rain['rain']>=500) & (rain['rain']<2000)].value_counts())
        }
        # 乡镇个数统计
        town_group = rain.groupby(['Town'])['rain'].max()  
        dict_town = {'Town':town_group.index,'rain':town_group.values}
        town_range = pd.DataFrame(data =dict_town)
        town_range['rank']=pd.cut(town_range['rain'],bins,right=False,labels=labels) 
        rank_town = {
            '≥0毫米':len(town_range[(town_range['rain']>0) & (town_range['rain']<30)].value_counts()),
            '≥30毫米':len(town_range[(town_range['rain']>=30) & (town_range['rain']<40)].value_counts()),
            '≥40毫米':len(town_range[(town_range['rain']>=40) & (town_range['rain']<50)].value_counts()),
            '≥50毫米':len(town_range[(town_range['rain']>=50) & (town_range['rain']<80)].value_counts()),
            '≥80毫米':len(town_range[(town_range['rain']>=80) & (town_range['rain']<100)].value_counts()),
            '≥100毫米':len(town_range[(town_range['rain']>=100) & (town_range['rain']<150)].value_counts()),
            '≥150毫米':len(town_range[(town_range['rain']>=150) & (town_range['rain']<200)].value_counts()),
            '≥200毫米':len(town_range[(town_range['rain']>=200) & (town_range['rain']<300)].value_counts()),
            '≥300毫米':len(town_range[(town_range['rain']>=300) & (town_range['rain']<500)].value_counts()),
            '≥500毫米':len(town_range[(town_range['rain']>=500) & (town_range['rain']<2000)].value_counts())
        }
        rain_dir = {
            "average_city":cnty,
            "station_index":rain_max,
            "hour_max":rain_hour,
            "town_max":town_max_str,
            "station_count":rank_station,
            "town_count":rank_town        
        }
        return rain_dir
    def text_wind(self):
        data = self.data
        bins=[0,10.7,13.8,17.1,20.7,24.4,28.4,32.6,36.9,41.4,46.1,50.9,56,80]
        labels=['6级风以下','6级风','7级风','8级风','9级风','10级风','11级风','12级风','13级风','14级风','15级风','16级风','17级风']
        wind = data[(data['wind']>10.7) & (data['wind']<5009)]
        wind['rank']=pd.cut(wind['wind'],bins,right=False,labels=labels)
        del wind['Unnamed: 0'] # 天擎不需要
        wind.reset_index(drop=True)# 天擎不需要
        wind_max = wind.sort_values(by="wind",ascending=False)
        rank_station = {
            '6级风以下':len(wind[(wind['wind']>0) & (wind['wind']<10.7)].to_dict()),
            '6级风':len(wind[(wind['wind']>=10.7) & (wind['wind']<13.8)].value_counts()),
            '7级风':len(wind[(wind['wind']>=13.8) & (wind['wind']<17.1)].value_counts()),
            '8级风':len(wind[(wind['wind']>=17.1) & (wind['wind']<20.7)].value_counts()),
            '9级风':len(wind[(wind['wind']>=20.7) & (wind['wind']<24.4)].value_counts()),
            '10级风':len(wind[(wind['wind']>=24.4) & (wind['wind']<28.4)].value_counts()),
            '11级风':len(wind[(wind['wind']>=28.4) & (wind['wind']<32.6)].value_counts()),
            '12级风':len(wind[(wind['wind']>=32.6) & (wind['wind']<36.9)].value_counts()),
            '13级风':len(wind[(wind['wind']>=36.9) & (wind['wind']<41.4)].value_counts()),
            '14级风':len(wind[(wind['wind']>=41.4) & (wind['wind']<46.1)].value_counts()),
            '15级风':len(wind[(wind['wind']>=46.1) & (wind['wind']<50.9)].value_counts()),
            '16级风':len(wind[(wind['wind']>=50.9) & (wind['wind']<56.4)].value_counts()),
            '17级及以上风':len(wind[(wind['wind']>=56.4) & (wind['wind']<80)].value_counts())
        }
        if rank_station['17级及以上风'] >0 :
            text = "【风力通报】 全市出现17级及以上大风，风力较大的有："
        elif rank_station['16级风'] >0 :
            text = "【风力通报】 全市出现16级及以上大风，风力较大的有："
        elif rank_station['15级风'] >0 :
            text = "【风力通报】 全市出现15级及以上大风，风力较大的有："
        elif rank_station['14级风'] >0 :
            text = "【风力通报】 全市出现14级及以上大风，风力较大的有："
        elif rank_station['13级风'] >0 :
            text = "【风力通报】 全市出现13级及以上大风，风力较大的有："
        elif rank_station['12级风'] >0 :
            text = "【风力通报】 全市出现12级及以上大风，风力较大的有："
        elif rank_station['11级风'] >0 :
            text = "【风力通报】 全市出现11级及以上大风，风力较大的有："
        elif rank_station['10级风'] >0 :
            text = "【风力通报】 全市出现10级及以上大风，风力较大的有："
        elif rank_station['9级风'] >0 :
            text = "【风力通报】 全市出现7～9级大风，风力较大的有："
        elif rank_station['8级风'] >0 :
            text = "【风力通报】 全市出现6～8级大风，风力较大的有："
        else:
            text = ""
        return rank_station
    def text_tmax(self):
        bins=[-40,35,37,40,80]
        labels=['正常','35度以上','37度以上','40度以上']
        tmax = data[(data['tmax']>-40) & (data['tmax']<5009)]
        tmax['rank']=pd.cut(tmax['tmax'],bins,right=False,labels=labels)
        del tmax['Unnamed: 0'] # 天擎不需要
        tmax.reset_index(drop=True)# 天擎不需要
        tmax_max = tmax.sort_values(by="tmax",ascending=False).head(10).to_dict()
        rank_tmax = {
           '正常':len(tmax[(tmax['tmax']>-40) & (tmax['tmax']<35)].to_dict()),
            '35度以上':len(tmax[(tmax['tmax']>=35) & (tmax['tmax']<37)].value_counts()),
            '37度以上':len(tmax[(tmax['tmax']>=37) & (tmax['tmax']<40)].value_counts()),
            '40度以上':len(tmax[(tmax['tmax']>=40) & (tmax['tmax']<80)].value_counts()) 
        }
        return rank_tmax
    def text_tmin(self):
        bins=[-20,-5,0,2,4]
        labels=['正常','35度以上','37度以上','40度以上']
        tmin = data[(data['tmin']>-40) & (data['tmin']<5009)]
        tmin['rank']=pd.cut(tmin['tmin'],bins,right=False,labels=labels)
        del tmin['Unnamed: 0'] # 天擎不需要
        tmin.reset_index(drop=True)# 天擎不需要
        tmin_max = tmin.sort_values(by="tmin").head(10).to_dict()
        rank_tmin = {
           '正常':len(tmin[(tmin['tmin']>-40) & (tmin['tmin']<35)].to_dict()),
            '35度以上':len(tmin[(tmin['tmin']>=35) & (tmin['tmin']<37)].value_counts()),
            '37度以上':len(tmin[(tmin['tmin']>=37) & (tmin['tmin']<40)].value_counts()),
            '40度以上':len(tmin[(tmin['tmin']>=40) & (tmin['tmin']<80)].value_counts()) 
        }
        return rank_tmin
    def text_view(self):
        bins=[0,50,200,500,99999]
        labels=['强浓雾','浓雾','大雾','正常']
        view = data[(data['view']>-40) & (data['view']<5009)]
        view['rank']=pd.cut(view['view'],bins,right=False,labels=labels)
        del view['Unnamed: 0'] # 天擎不需要
        view.reset_index(drop=True)# 天擎不需要
        view_max = view.sort_values(by="view",ascending=False).head(10).to_dict()
        rank_view = {
           '强浓雾':len(view[(view['view']>0) & (view['view']<50)].to_dict()),
            '浓雾':len(view[(view['view']>=50) & (view['view']<200)].value_counts()),
            '大雾':len(view[(view['view']>=200) & (view['view']<500)].value_counts()),
            '正常':len(view[(view['view']>=500) & (view['view']<99990)].value_counts()) 
        }
        return rank_view

# 气象快报
city_code = "331000"
start = '2023-07-01 20:52:00'
end = '2023-07-02 20:52:00'
worker = station_zdz(city_code,start,end)
worker.text_wind()